In [1]:
import tensorflow as tf

In [2]:
file_url = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'

In [3]:
zip_dir = tf.keras.utils.get_file('cats_and_dogs.zip', origin=file_url, extract=True)

68608000/68606236 [==============================] - 0s 0us/step


In [4]:
import pathlib

In [5]:
path = pathlib.Path(zip_dir).parent / 'cats_and_dogs_filtered'

In [6]:
train_dir = path / 'train'
validation_dir = path / 'validation'

In [9]:
total_train = 2000
total_val = 1000

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [11]:
train_image_generator = ImageDataGenerator(rescale=1./255)
validation_image_generator = ImageDataGenerator(rescale=1./255)

In [12]:
batch_size=32
img_height = 224
img_width = 224

In [13]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(img_height, img_width),
                                                           class_mode='binary')

Found 2000 images belonging to 2 classes.


In [14]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=validation_dir,
                                                              target_size=(img_height, img_width),
                                                              class_mode='binary')

Found 1000 images belonging to 2 classes.


In [15]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

In [16]:
np.random.seed(8)
tf.random.set_seed(8)

In [17]:
import tensorflow_hub as hub

In [18]:
MODULE_HANDLE = 'https://tfhub.dev/google/efficientnet/b0/feature-vector/1'
module = hub.load(MODULE_HANDLE)

In [19]:
model = tf.keras.Sequential([
        hub.KerasLayer(MODULE_HANDLE,input_shape=(224, 224, 3)),
        layers.Dense(500, activation='relu'),
        layers.Dense(1, activation='sigmoid')
])

In [20]:
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.001), metrics=['accuracy'])

In [21]:
model.fit(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=5,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size
)

Epoch 1/5
125/125 [==============================] - 12s 92ms/step - loss: 0.0504 - accuracy: 0.9810 - val_loss: 0.0196 - val_accuracy: 0.9929
Epoch 2/5
125/125 [==============================] - 11s 87ms/step - loss: 0.0126 - accuracy: 0.9960 - val_loss: 0.0248 - val_accuracy: 0.9909
Epoch 3/5
125/125 [==============================] - 11s 86ms/step - loss: 0.0064 - accuracy: 0.9975 - val_loss: 0.0265 - val_accuracy: 0.9899
Epoch 4/5
125/125 [==============================] - 11s 87ms/step - loss: 0.0026 - accuracy: 0.9990 - val_loss: 0.0251 - val_accuracy: 0.9899
Epoch 5/5
125/125 [==============================] - 11s 87ms/step - loss: 4.9526e-04 - accuracy: 1.0000 - val_loss: 0.0291 - val_accuracy: 0.9899
